In [1]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import pandas as pd
from functions import join_stocks_crypto, generate_rand_portfolios, sharpe_ratio_calculation, select_top_five, run_clustering_model
from functions_post_clustering import reoptimize_weights, select_complementary_cryptos, supplement_set_with_cryptos

import cvxpy as cp
import random
import json
import warnings

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
df_all_stocks = pd.read_csv('stocks_data_filled.csv',index_col='Date')
cryptos_df = pd.read_csv('cryptos_data.csv', index_col='Date')
joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left')

cryptos_list = list(cryptos_df.columns)

In [3]:
tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)


#Select top five sharpe ratio portfolios from a portfolio
sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)
top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [4]:
#Optimize Traditional Portfolios
# for i in range(999, 1001):

#     print('Doing', i)

#     top_five_sets = dict(itertools.islice(top_five_dict.items(), i, i+1))
#     results = run_min_variance(df_all_stocks, top_five_sets, min_weight_for_top_five=0.05)  #TRY DIFFERENT WEIGHTS FOR top_five
#     with open(f"min_variance_portfolio_jsons/my_dict{i}.json", "w") as f:
#         json.dump(results, f, indent=4)




#Reassemble the results of the optimization - jsons
min_var_portfolios = dict()
for i in range(1,1000):
    with open(f'min_variance_portfolio_jsons/my_dict{i}.json') as f:
        port = json.load(f)
        min_var_portfolios.update(port)

# with open(f"full_optimized_min_variance.json", "w") as f:
#     json.dump(min_var_portfolios, f, indent=4)

In [5]:
# top_five = top_five_dict['portfolio_0']
# top_five

In [6]:
#Actually Cluster the stocks

warnings.simplefilter(action='ignore', category=FutureWarning) #supress warnings for cleanliness
warnings.simplefilter(action='ignore', category=RuntimeWarning)

joined_df.index = pd.to_datetime(joined_df.index)
joined_df_weekly = joined_df.resample('W').last() #try aggregating on a weekly level
joined_df_3days = joined_df.resample('3D').last()

#AHC
df = joined_df_3days.rolling(window=60, center=True).mean()
labels, tickers_with_labels_ahc, _, _ = run_clustering_model(df, n_clus=7, model_name='ahc', linkage='complete', return_mode='geometric', n_init=3)

#KMEANS
df = joined_df_3days.rolling(window=60, center=True).mean()
labels, tickers_with_labels_kmeans, _, _ = run_clustering_model(df, n_clus=7, model_name='kmeans', linkage=None, return_mode='geometric', n_init=3)

#KSHAPE
df = joined_df.rolling(window=7, center=True).mean()
labels, tickers_with_labels_kshape, _, _ = run_clustering_model(df, n_clus=6, model_name='kmeans', linkage=None, return_mode='geometric', n_init=3)

In [7]:
#Random Portfolios supplemented with cryptos
random_w_cryptos_kmeans = supplement_set_with_cryptos(portfolio_set=random_portfolios, cryptos_list=cryptos_list, tickers_with_labels=tickers_with_labels_kmeans, df_prices=joined_df, n_cryptos=3)
random_w_cryptos_kshape = supplement_set_with_cryptos(portfolio_set=random_portfolios, cryptos_list=cryptos_list, tickers_with_labels=tickers_with_labels_kshape, df_prices=joined_df, n_cryptos=3)
random_w_cryptos_ahc = supplement_set_with_cryptos(portfolio_set=random_portfolios, cryptos_list=cryptos_list, tickers_with_labels=tickers_with_labels_ahc, df_prices=joined_df, n_cryptos=3)

random_w_cryptos_random = supplement_set_with_cryptos(portfolio_set=random_portfolios, cryptos_list=cryptos_list, tickers_with_labels=None, df_prices=joined_df, n_cryptos=3, random_alloc=True)

#MinVar Portfolios supplemented with cryptos
minvar_w_cryptos_kmeans = supplement_set_with_cryptos(portfolio_set=min_var_portfolios, cryptos_list=cryptos_list, tickers_with_labels=tickers_with_labels_kmeans, df_prices=joined_df, n_cryptos=3)
minvar_w_cryptos_kshape = supplement_set_with_cryptos(portfolio_set=min_var_portfolios, cryptos_list=cryptos_list, tickers_with_labels=tickers_with_labels_kshape, df_prices=joined_df, n_cryptos=3)
minvar_w_cryptos_ahc = supplement_set_with_cryptos(portfolio_set=min_var_portfolios, cryptos_list=cryptos_list, tickers_with_labels=tickers_with_labels_ahc, df_prices=joined_df, n_cryptos=3)

minvar_w_cryptos_random = supplement_set_with_cryptos(portfolio_set=min_var_portfolios, cryptos_list=cryptos_list, tickers_with_labels=None, df_prices=joined_df, n_cryptos=3, random_alloc=True)


In [8]:
#Random Portfolios supplemented with cryptos reoptimized
random_w_cryptos_kmeans_equalw = reoptimize_weights(df_prices=joined_df, portfolio_set=random_w_cryptos_kmeans, how='equal_weights', min_weight=0.02, rf_rate=0.02)
random_w_cryptos_kmeans_maxsharpe = reoptimize_weights(df_prices=joined_df, portfolio_set=random_w_cryptos_kmeans, how='max_sharpe', min_weight=0.02)

random_w_cryptos_kshape_equalw  = reoptimize_weights(df_prices=joined_df, portfolio_set=random_w_cryptos_kshape, how='equal_weights', min_weight=0.02)
random_w_cryptos_kshape_maxsharpe = reoptimize_weights(df_prices=joined_df, portfolio_set=random_w_cryptos_kshape, how='max_sharpe', min_weight=0.02)

random_w_cryptos_ahc_equalw  = reoptimize_weights(df_prices=joined_df, portfolio_set=random_w_cryptos_ahc, how='equal_weights', min_weight=0.02)
random_w_cryptos_ahc_maxsharpe = reoptimize_weights(df_prices=joined_df, portfolio_set=random_w_cryptos_ahc, how='max_sharpe', min_weight=0.02)

random_w_cryptos_random_equalw  = reoptimize_weights(df_prices=joined_df, portfolio_set=random_w_cryptos_random, how='equal_weights', min_weight=0.02)
random_w_cryptos_random_maxsharpe = reoptimize_weights(df_prices=joined_df, portfolio_set=random_w_cryptos_random, how='max_sharpe', min_weight=0.02)

#MinVar Portfolios supplemented with cryptos reoptimized
minvar_w_cryptos_kmeans_equalw = reoptimize_weights(df_prices=joined_df, portfolio_set=minvar_w_cryptos_kmeans, how='equal_weights', min_weight=0.02, rf_rate=0.02)
minvar_w_cryptos_kmeans_maxsharpe = reoptimize_weights(df_prices=joined_df, portfolio_set=minvar_w_cryptos_kmeans, how='max_sharpe', min_weight=0.02)

minvar_w_cryptos_kshape_equalw  = reoptimize_weights(df_prices=joined_df, portfolio_set=minvar_w_cryptos_kshape, how='equal_weights', min_weight=0.02)
minvar_w_cryptos_kshape_maxsharpe = reoptimize_weights(df_prices=joined_df, portfolio_set=minvar_w_cryptos_kshape, how='max_sharpe', min_weight=0.02)

minvar_w_cryptos_ahc_equalw  = reoptimize_weights(df_prices=joined_df, portfolio_set=minvar_w_cryptos_ahc, how='equal_weights', min_weight=0.02)
minvar_w_cryptos_ahc_maxsharpe = reoptimize_weights(df_prices=joined_df, portfolio_set=minvar_w_cryptos_ahc, how='max_sharpe', min_weight=0.02)

minvar_w_cryptos_random_equalw  = reoptimize_weights(df_prices=joined_df, portfolio_set=minvar_w_cryptos_random, how='equal_weights', min_weight=0.02)
minvar_w_cryptos_random_maxsharpe = reoptimize_weights(df_prices=joined_df, portfolio_set=minvar_w_cryptos_random, how='max_sharpe', min_weight=0.02)






c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\cvxpy\problems\problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\cvxpy\problems\problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\cvxpy\problems\problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\cvxpy\problems\problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for mor

In [9]:
all_sets = {'random_w_cryptos_kmeans_equalw': random_w_cryptos_kmeans_equalw,
            'random_w_cryptos_kmeans_maxsharpe': random_w_cryptos_kmeans_maxsharpe,
            'random_w_cryptos_kshape_equalw': random_w_cryptos_kshape_equalw,
            'random_w_cryptos_kshape_maxsharpe': random_w_cryptos_kshape_maxsharpe,
            'random_w_cryptos_ahc_equalw': random_w_cryptos_ahc_equalw,
            'random_w_cryptos_ahc_maxsharpe': random_w_cryptos_ahc_maxsharpe,
            'random_w_cryptos_random_equalw': random_w_cryptos_random_equalw,
            'random_w_cryptos_random_maxsharpe': random_w_cryptos_random_maxsharpe,
            'minvar_w_cryptos_kmeans_equalw': minvar_w_cryptos_kmeans_equalw,
            'minvar_w_cryptos_kmeans_maxsharpe': minvar_w_cryptos_kmeans_maxsharpe, 
            'minvar_w_cryptos_kshape_equalw': minvar_w_cryptos_kshape_equalw,
            'minvar_w_cryptos_kshape_maxsharpe': minvar_w_cryptos_kshape_maxsharpe,
            'minvar_w_cryptos_ahc_equalw': minvar_w_cryptos_ahc_equalw,
            'minvar_w_cryptos_ahc_maxsharpe': minvar_w_cryptos_ahc_maxsharpe,
            'minvar_w_cryptos_random_equalw': minvar_w_cryptos_random_equalw,
            'minvar_w_cryptos_random_maxsharpe': minvar_w_cryptos_random_maxsharpe}

with open(f"all_optimized_sets_for_simulation.json", "w") as f:
    json.dump(all_sets, f, indent=4)